### Teste 3

Deseja-se prospectar empresas que possuam soluções em *tratamento de água*, principalmente relativas à: *solutions on waste and water, Improve water quality and water efficiency use, water contamination, water for human consumption, water resources*

### Imports

In [1]:
import re
import string
import pandas as pd

import nltk

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import gensim

In [2]:
try:
    len(stopwords.words('english'))
except:
    nltk.download('stopwords')

In [3]:
def ajuste_col_coord(s: pd.Series) -> pd.Series:
    '''
    Corrige leitura da coluna de latitude e longitude dos dados amostra_canada.csv
    Input
    -----
    s: pd.Series com dtype str e registros errados no formato "<coordenada>;;;;;;;"
    Output
    ------
    pd.Series com dtype float de mesma dimensão de `s`, mantendo somente dígitos,
    "." e "-", para fazer jus a uma coordenada geográfica.
    '''
    return (
        s.str.replace('[^\\d.-]', '', regex=True) #remove tudo que nao eh um digito ou "." ou "-"
        .replace('',None)
        .astype(float)
    )

In [4]:
def remove_pontuacao(sentence: str) -> str:
    return sentence.translate(str.maketrans({x: '' for x in string.punctuation}))

def remove_numeros(sentence: str) -> str:
    return re.sub('[0-9]+', '', sentence)

def preprocess(sentence: str) -> str:
    sentence = sentence.lower()
    sentence = remove_pontuacao(sentence)
    sentence = remove_numeros(sentence)
    tokenizer = RegexpTokenizer(r'\w+')
    
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]

    return filtered_words

### Leitura dataset

In [5]:
n_base = pd.read_csv('Planilha.csv', sep=',', usecols=[0] ).shape[0]

In [6]:
df = pd.read_csv(
    'Planilha.csv', 
    sep=',', 
    on_bad_lines='skip'
)

In [7]:
print(f'Quantidade de registros tabela completa: `{n_base}` quantidade lida ignorando registros fora do padrão: `{df.shape[0]}`')

Quantidade de registros tabela completa: `21299` quantidade lida ignorando registros fora do padrão: `20640`


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   20640 non-null  object
 1   description            3701 non-null   object
 2   employees              3561 non-null   object
 3   total_funding          504 non-null    object
 4   city                   3699 non-null   object
 5   subcountry             3699 non-null   object
 6   lat                    3699 non-null   object
 7   lng;;;;;;;;;;;;;;;;;;  3543 non-null   object
dtypes: object(8)
memory usage: 1.3+ MB


In [9]:
df.columns = [x.replace(';','') for x in df.columns]

In [10]:
df[df['lng'].notna()].sample(5)

,name,description,employees,total_funding,city,subcountry,lat,lng
1365,sweat free,we are a health tech company solving the probl...,0.0,NaN,waterloo,ontario,43.466874,-80.524635;;;;;;;;;;;;;;;;;;
14155,dynamic legal solutions â€“ find criminal defe...,dynamic legal is a company in ottawa devoted t...,1.0,NaN,toronto,ontario,43.70011,-79.4163;;;;;;;;;;;;;;;;;;
11462,zenith marketing &amp; business development,one agency for all your marketing and business...,0.0,-,langley,british columbia,49.10647185,-122.550131156815;;;;;;;;;;;;;;;;;
6048,frank lending,the loan is a lending platform that offers loa...,1.0,NaN,toronto,ontario,43.700109999999995,-79.4163;;;;;;;;;;;;;;;;;;
20438,sprott money,sprott money is a leading online precious meta...,50.0,NaN,toronto,ontario,43.70011,-79.4163;;;;;;;;;;;;;;;;;


In [11]:
df['lng'] = ajuste_col_coord(df['lng'])
df['lat'] = ajuste_col_coord(df['lat'])

In [12]:
df[['lat','lng']].describe()

,lat,lng
count,3699.000000,3541.000000
mean,40.069262,-111.995912
std,28.053005,1237.018224
min,-123.369300,-73692.000000
25%,43.700110,-114.085290
50%,43.700110,-79.416300
75%,49.249660,-79.416300
max,56.726760,-52.709310


Quantidade de descrições das empresas muito abaixo do total. Verificando problema na leitura:

In [13]:
df[df['description'].isna()].shape

(16939, 8)

#### Tratamento de registros com descrição não lida corretamente

In [14]:
errados = df[df['description'].isna()]

In [15]:
errados.head(3)

,name,description,employees,total_funding,city,subcountry,lat,lng
0,"fitin,""fitin is the latest fit tech platform t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"quinditech,""our services include independent &...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"botfirst,""chatlanding is a digital chatbot pla...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
ex_errado_teste = errados.name.values[0]

In [17]:
ex_errado_teste

'fitin,"fitin is the latest fit tech platform that provides a professional marketing and booking tool for gyms, studios, independent instructors and personal trainers. customers want a combination of \'trip advisor\' and \'groupon\', where they can read reviews, check out schedules, browse photos, and bookâ€¦all in one place. gyms and trainers want to be able to fill their classes and schedules. at fitin, we want to connect them and provide the community with a self-sustaining resource that helps everyone. with the wide variety of fitness entrepreneurs out there, we have amazing diversity in our workout options, but they are currently too difficult to find, if you donâ€™t already know where to go â€“ you need to search for hours, reach out to friends and family for referrals that may or may not be a good fit for you. we have over 700 classes already listed in our platform, and we want to be the resource that inspires canadians to just get movingâ€¦no matter what kind of class that may 

Os registros errados ficaram com suas descrições junto ao nome, e aparentemente estão com as aspas indicando o texto correto. 

Abaixo, vou verificar se quantos registros possuem somente 2 aspas, na esperança de que esse padrão de fato corresponda às descrições dessa tabela.

In [18]:
errados.name.map(lambda x: len([x for x in list(x) if x == '"'])).value_counts()

name
2     16048
6       634
10      132
14       45
4        39
8        12
18       11
0         5
26        4
22        3
1         2
12        2
16        1
30        1
Name: count, dtype: int64

A maioria possui somente 2 aspas. Vou usar isso para fazer o parsing dos registros em `errados`

In [19]:
# Criando booleano indicando os registros que possuem somente 2 aspas

errados['bool_2aspas'] = errados['name'] \
    .map(lambda x: len([x for x in list(x) if x == '"']) == 2)

C:\Users\eric.barbosa\AppData\Local\Temp\ipykernel_18492\695724686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errados['bool_2aspas'] = errados['name'] \


In [20]:
# Para todos os registros com 2 aspas, coletando descricao por meio do regex "<qualquer caracter>"

errados.loc[errados['bool_2aspas'], 'desc_aux'] = errados.loc[errados['bool_2aspas'], 'name'] \
    .map(lambda x: re.findall(pattern='".*"', string=x)[0].replace('"',''))

C:\Users\eric.barbosa\AppData\Local\Temp\ipykernel_18492\1042011603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errados.loc[errados['bool_2aspas'], 'desc_aux'] = errados.loc[errados['bool_2aspas'], 'name'] \


In [21]:
errados.head(2)

,name,description,employees,total_funding,city,subcountry,lat,lng,bool_2aspas,desc_aux
0,"fitin,""fitin is the latest fit tech platform t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,fitin is the latest fit tech platform that pro...
1,"quinditech,""our services include independent &...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,our services include independent & collaborati...


In [22]:
# Criando coluna com os dados restantes da coluna "name", substituindo a descrição por 'nan'

errados.loc[errados['bool_2aspas'], 'dados'] = (
    errados
    .loc[errados['bool_2aspas']]
    .apply(lambda _df: _df['name'].replace(_df['desc_aux'],'nan'), axis=1)
)

C:\Users\eric.barbosa\AppData\Local\Temp\ipykernel_18492\109269651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errados.loc[errados['bool_2aspas'], 'dados'] = (


In [23]:
# Splitando dados restantes em coluna com lista das informações

errados.loc[errados['bool_2aspas'], 'dados_split'] = (
    errados
    .loc[errados['bool_2aspas'], 'dados']
    .map(lambda x: x.split(','))
)

C:\Users\eric.barbosa\AppData\Local\Temp\ipykernel_18492\3647934005.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errados.loc[errados['bool_2aspas'], 'dados_split'] = (


In [24]:
# Nem todas as informações possuem os 8 campos após o split por ",". Criando coluna booleana que indica quais estão aderentes

errados.loc[errados['bool_2aspas'], 'dados_len8'] = (
    errados
    .loc[errados['bool_2aspas'], 'dados_split']
    .map(len) == 8
)

C:\Users\eric.barbosa\AppData\Local\Temp\ipykernel_18492\423011998.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errados.loc[errados['bool_2aspas'], 'dados_len8'] = (


#### Criando novo dataframe tratado

In [25]:
dfnew = (
    errados.loc[errados['bool_2aspas'] & errados['dados_len8'], ['dados_split']]
    .apply(lambda x: x[0], axis=1, result_type='expand')
)

dfnew.columns=  df.columns.tolist()

dfnew['description'] = errados.loc[errados['bool_2aspas'] & errados['dados_len8'], 'desc_aux']

dfnew['lat'] = ajuste_col_coord(dfnew['lat'])
dfnew['lng'] = ajuste_col_coord(dfnew['lng'])
dfnew['employees'] = dfnew['employees'].astype(float)

dfnew['total_funding'] = dfnew['total_funding'].str.replace('[^\d.]','',regex=True).replace('', None).astype(float)

dfnew = dfnew.reset_index().rename({'index': 'id'}, axis=1)

dfnew.head()

C:\Users\eric.barbosa\AppData\Local\Temp\ipykernel_18492\649374906.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda x: x[0], axis=1, result_type='expand')


,id,name,description,employees,total_funding,city,subcountry,lat,lng
0,0,fitin,fitin is the latest fit tech platform that pro...,1.0,NaN,toronto,ontario,43.70011,-79.41630
1,1,quinditech,our services include independent & collaborati...,10.0,NaN,montreal,quebec,45.50884,-73.58781
2,2,botfirst,chatlanding is a digital chatbot platform wher...,1.0,NaN,toronto,ontario,43.70011,-79.41630
3,4,stylify,stylify is a mobile platform that connects peo...,1.0,NaN,toronto,ontario,43.70011,-79.41630
4,6,weave travels,weave is a web platform for travel companies. ...,1.0,NaN,toronto,ontario,43.70011,-79.41630


### Análise das descrições das empresas

In [26]:
descricoes = dfnew.description.map(preprocess).tolist()

In [27]:
query_empresa = 'solutions on waste and water, Improve water quality and water efficiency use, water contamination, water for human consumption, water resources'
query_prep = preprocess(query_empresa)

In [28]:
# Adequando inputs para entrada no modelo Doc2Vec

documents = [gensim.models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(descricoes)]


# Aplicando modelo doc2vec com embedding de tamanho 100
model = gensim.models.doc2vec.Doc2Vec(
    documents, 
    vector_size=100, 
    window=2, 
    min_count=1, 
    workers=2
)

In [29]:
# Exemplo de embedding encontrado

model.wv['water']

array([-1.2099103 ,  0.20488884,  1.6075598 ,  0.60702103,  0.4861572 ,
       -0.13249396, -0.08792464,  0.14884694,  0.35954314, -0.38573462,
        0.27761355, -0.7246184 , -0.4033357 , -0.20843267, -0.02686915,
       -0.43054634,  0.15667634, -1.1043056 , -0.04929312, -0.94657904,
       -1.5428348 ,  0.41827106,  1.1029675 ,  0.6470845 , -0.8495491 ,
        0.513555  ,  0.04651935, -0.0699788 , -0.90964514, -0.00818379,
       -0.08633345, -0.8558223 ,  0.11597276,  0.5617644 ,  0.6431451 ,
        0.17407823, -0.49140093, -0.35314494, -0.08184387, -0.4925411 ,
       -0.44056627, -0.00377058, -0.63031465, -0.52892715,  0.30026174,
       -0.35715178, -0.4320588 , -0.5256435 ,  0.1336307 ,  0.83868235,
        0.8164649 ,  0.03321948,  0.05149238,  0.6524719 , -0.7681994 ,
        0.48252478,  0.8543782 , -0.08985571, -0.2594493 , -0.6908234 ,
       -1.6369907 , -0.33920783, -1.5928137 , -0.24471192, -0.09950631,
        0.0152641 , -0.2330703 ,  0.207311  , -0.20817344,  0.80

In [30]:
from typing import List

def similaridade(
        query: str, 
        model: gensim.models.doc2vec.Doc2Vec, 
        n_docs: int, 
        company: pd.DataFrame,
        tagged_documents: List[gensim.models.doc2vec.TaggedDocument]
) -> pd.DataFrame:
    '''
    
    '''
    query_prep = preprocess(query)
     
    query_emb = model.infer_vector(query_prep)

    ret = pd.DataFrame(model.dv.most_similar([query_emb], topn=n_docs), columns=['index','similarity'])

    tagget_ret = pd.DataFrame([
        (x.tags[0], ' '.join(x.words)) 
        for x in tagged_documents 
        if x.tags[0] in ret['index'].tolist()
    ], columns=['index','desc'])

    return pd.merge(
        company[['id','name']].rename({'id': 'index'}, axis=1),
        tagget_ret, on='index'
    ).merge(ret, on='index')

In [33]:
resultado_teste = similaridade(
    query_empresa,
    model=model,
    n_docs=10,
    company=dfnew[['id','name']],
    tagged_documents=documents
)

In [36]:
resultado_teste.sort_values(by='similarity', ascending=False)

,index,name,desc,similarity
0,113,advanced analytics and research lab,take recycled materials automotive waste inclu...,0.847569
8,11277,innovation centric group,livestock water recycling worldâ leading provi...,0.790897
2,2326,butterfl.ai,homebased scientifictechnological solution mak...,0.789645
1,1868,just booked,bluedot strives make world safer place infecti...,0.782513
9,12127,glacier lake resources inc.,data sciences corp medical technology data sci...,0.780849
5,5756,sage labs,inflatable grow bed systems form channel struc...,0.778983
6,7070,centschange.com,planetecg pivoting new strategy era big data i...,0.778175
7,10190,fusionpipe software,blue skyâ association harbour air started aim ...,0.773585
4,4029,mistplay,hnano provides solutions environmental challen...,0.771325
3,2499,six club,natural source enhances corporate culture prov...,0.770407
